# Apps using stackview in combination with Voila
If you combine stackview with [voila](https://voila.readthedocs.io/en/stable/), you can build powerfull image processing apps and demonstrators. 
Just build a graphical user interface in a Jupyter notebook and then start it using
```
voila apoc_gui.ipynb
```

In [1]:
from skimage.io import imread
import numpy as np
import stackview
import ipywidgets as widgets
from IPython.display import display, clear_output
import apoc
import pyclesperanto_prototype as cle
import time

In [2]:
#from skimage.data import cells3d
#image = cells3d()[:,1]
image = imread('data/blobs.tif')
#stackview.slice(image)

In [3]:
annotation = cle.push(np.zeros_like(image, dtype=np.uint32)).astype(np.uint32)
labels = cle.push(np.zeros_like(image, dtype=np.uint16)).astype(np.uint32)

In [4]:
# Buttons
train_button = widgets.ToggleButton(value=True, 
                                    description='Train', 
                                    button_style='success', 
                                    tooltip='Start machine learning training')
train_button.layout.height = '500'

def on_train_click(change):
    if change['new'] == True:
        model_filename = "object_segmenter.cl"
        apoc.erase_classifier(model_filename)
        model = apoc.ObjectSegmenter(model_filename)
        model.train(apoc.PredefinedFeatureSet.medium_quick.value, image=image, ground_truth=annotation)
        new_labels = model.predict(image=image)
        cle.copy(new_labels, labels)
        slice_placeholder.update()
        time.sleep(0.5)
        slice_placeholder.update()

train_button.observe(on_train_click, names='value')

reset_button = widgets.ToggleButton(value=True, 
                                    description='Reset', 
                                    button_style='danger', 
                                    tooltip='Reset annotation')

def on_reset_click(change):
    if change['new'] == True:
        cle.set(annotation, 0)
        cle.set(labels, 0)        
        annotate_placeholder.update()
        slice_placeholder.update()
        # not sure why this is necessary
        time.sleep(0.5)
        annotate_placeholder.update()
        slice_placeholder.update()
        
reset_button.observe(on_reset_click, names='value')

# Image interaction
annotate_placeholder = stackview.annotate(image, annotation)
slice_placeholder = stackview.curtain(image, labels, alpha=0.7)

# Initially display the annotate field
display(widgets.HBox([
    widgets.VBox([
        annotate_placeholder, 
        widgets.HBox([
            reset_button,
            train_button])
        ]),
    slice_placeholder
]))

